### Setup

In [1]:
""" Python Standard Library """
import os
import datetime as dt
import json
import logging
""" Third Party Imports """
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
%matplotlib inline
import seaborn as sns
""" Local Imports """
if 'requirements.txt' not in os.listdir(os.getcwd()):
    os.chdir(os.path.join('..','..','..'))
print(os.getcwd())
from covid_model.model import CovidModel
from covid_model.runnable_functions import do_single_fit, do_fit_scenarios, do_create_report #do_create_multiple_reports
from covid_model.utils import setup, get_filepath_prefix
from covid_model.analysis.charts import plot_transmission_control

os.environ['gcp_project'] = 'co-covid-models'
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = "co-covid-models-credentials.json"

/Users/emwu9912/PycharmProjects/covid-models


In [2]:
# set up the output directory for this Jupyter notebook
outdir = setup("20221202_scenario_projections.ipynb")

2022/11/30 17:14:22 INFO|============================================================


### Fit an initial scenario through present day

In [3]:
# designate the arguments for how the model will behave
model_args = {
    'params_defs': 'covid_model/analysis/20221109_nov_gov_briefing/20221109_scenario_params.json',
    'region_defs': 'covid_model/input/region_definitions.json',
    'vacc_proj_params': 'covid_model/analysis/20221109_nov_gov_briefing/20221109_vacc_proj_params.json',
    'start_date': '2020-01-24',
    'end_date': '2024-01-01',
}

# this is how the fit will behave
# place the outdir argument here to tell the model fit where to go
fit_args = {'outdir': outdir,
            'fit_end_date': '2022-11-29'
}

# because all the scenarios are the same 
model = do_single_fit(**fit_args, **model_args)

2022/11/30 17:14:32 INFO|-------------------------|{} Retrieving vaccinations data
2022/11/30 17:14:34 INFO|-------------------------|{} Constructing vaccination projections
2022/11/30 17:14:36 INFO|-------------------------|{} Retrieving hospitalizations data
2022/11/30 17:14:39 INFO|----------------------|{} Prepping Model
2022/11/30 17:14:39 INFO|-----------------------|{} Prepping Model
2022/11/30 17:16:04 INFO|----------------------|{} Model prepped for fitting in 85.237903792 seconds.
2022/11/30 17:16:04 INFO|----------------------|{} Will fit 35 times
2022/11/30 17:17:15 INFO|----------------------|{'fit_batch': '0'}: Transmission control fit 1/35 completed in 71.474012125 seconds: {0: {'co': 0.8306366879529625}, 14: {'co': 0.6980074624702209}, 28: {'co': 0.5697465921667575}, 42: {'co': 0.002835705861103618}, 56: {'co': 0.6814697431114114}, 70: {'co': 0.891275392270152}}
2022/11/30 17:17:15 INFO|-----------------------|{'fit_batch': '0'}: Running forward sim
2022/11/30 17:17:32 

2022/11/30 17:22:15 INFO|----------------------|{'fit_batch': '18'}: Transmission control fit 19/35 completed in 9.297620750000021 seconds: {504: {'co': 0.792761487613059}, 518: {'co': 0.7862978568792409}, 532: {'co': 0.7661786118953587}, 546: {'co': 0.7176009783156061}, 560: {'co': 0.7506313149355148}, 574: {'co': 0.7457868883128519}}
2022/11/30 17:22:15 INFO|-----------------------|{'fit_batch': '18'}: Running forward sim
2022/11/30 17:22:31 INFO|----------------------|{'fit_batch': '19'}: Transmission control fit 20/35 completed in 14.10272304199998 seconds: {532: {'co': 0.7655333909968927}, 546: {'co': 0.7194148624553423}, 560: {'co': 0.7472635853360977}, 574: {'co': 0.7558547283666177}, 588: {'co': 0.792687453499594}, 602: {'co': 0.7730462164955765}}
2022/11/30 17:22:31 INFO|-----------------------|{'fit_batch': '19'}: Running forward sim
2022/11/30 17:22:46 INFO|----------------------|{'fit_batch': '20'}: Transmission control fit 21/35 completed in 13.689633999999955 seconds: {56

2022/11/30 17:27:25 INFO|-----------------------|{'fit_batch': '34', 'run_type': 'fit'}: Running forward sim
2022/11/30 17:27:27 INFO|----------------------|{'fit_batch': '34', 'run_type': 'fit'}: Uploading final results
2022/11/30 17:27:36 INFO|----------------------|{'fit_batch': '34', 'run_type': 'fit'}: spec_id: 4515


### Run the model

In [ ]:
# if you want to pass an old fit spec ID through, you can run this code so you don't have to refit the whole model
# otherwise, ignore
#model_args = {
    #'base_spec_id': 4509, # use the spec id that was output from the model fit
#}
#model = CovidModel(**model_args)
#model.prep()

In [4]:
# now create the reports for the model, indexing through all the values of TC we want to simulate
logging.info('Projecting')
for tc in [0.7160, 0.7955, 0.8751]:
    logging.info('')
    model.update_tc({model.date_to_t("2022-11-29"): {'co': tc}}, replace = False)
    model.tags.update({'tc': tc})
    model.solve_seir()
    do_create_report(model, outdir=outdir, prep_model=False, solve_model=False, immun_variants=['ba45', 'emv'], from_date='2022-01-01')

    model.solution_sum_df(['seir', 'variant', 'immun']).unstack().to_csv(get_filepath_prefix(outdir, tags=model.tags) + 'states_seir_variant_immun_total_all_at_once_projection.csv')
    model.solution_sum_df().unstack().to_csv(get_filepath_prefix(outdir, tags=model.tags) + 'states_full_projection.csv')

    logging.info(f'{str(model.tags)}: Running forward sim')
    fig = plt.figure(figsize=(10, 10), dpi=300)
    ax = fig.add_subplot(211)
    hosps_df = model.modeled_vs_observed_hosps().reset_index('region').drop(columns='region')
    hosps_df.plot(ax=ax)
    ax.set_xlim(dt.datetime.strptime('2022-01-01', "%Y-%m-%d").date(), dt.datetime.strptime('2024-01-01', "%Y-%m-%d").date())
    ax = fig.add_subplot(212)
    plot_transmission_control(model, ax=ax)
    ax.set_xlim(dt.datetime.strptime('2022-01-01', "%Y-%m-%d").date(), dt.datetime.strptime('2024-01-01', "%Y-%m-%d").date())
    plt.savefig(get_filepath_prefix(outdir, tags=model.tags) + 'model_forecast.png')
    plt.close()
    hosps_df.to_csv(get_filepath_prefix(outdir, tags=model.tags) + '_model_forecast.csv')
    json.dump(model.tc, open(get_filepath_prefix(outdir, tags=model.tags) + 'model_forecast_tc.json', 'w'))

    logging.info('Running reports')

2022/11/30 17:46:42 INFO|Projecting
2022/11/30 17:46:42 INFO|
2022/11/30 17:46:49 WARNING|No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
2022/11/30 17:49:06 INFO|{'fit_batch': '34', 'run_type': 'fit', 'tc': 0.716}: Running forward sim
2022/11/30 17:49:07 INFO|Running reports
2022/11/30 17:49:07 INFO|
2022/11/30 17:49:13 WARNING|No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
2022/11/30 17:51:29 INFO|{'fit_batch': '34', 'run_type': 'fit', 'tc': 0.7955}: Running forward sim
2022/11/30 17:51:31 INFO|Running reports
2022/11/30 17:51:31 INFO|
2022/11/30 17:51:37 WARNING|No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
2022/11/30 17:53:52 INFO|{'fit_batch': '34', 'run_ty